<a href="https://colab.research.google.com/github/juelha/IANNWTF/blob/sabine/hw08_Sabine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras

In [2]:
# dataset and prepossing
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

shape_train = x_train.shape
shape_test = x_test.shape

x_train = tf.data.Dataset.from_tensor_slices(x_train)
x_test = tf.data.Dataset.from_tensor_slices(x_test)
print(x_train)

def preprocessing(tensor):
  tensor = tensor.map(lambda images: (tf.cast(images, tf.float32) / 255.)) # normalise
  tensor = tensor.map(lambda images: (tf.expand_dims(images, -1))) # make 3D in case
  tensor = tensor.map(lambda images: (tf.add(images, tf.random.normal((28,28,1), mean=0, stddev=1)))) # add noise with random 28,28 matrix 
  tensor = tensor.map(lambda images: (tf.clip_by_value(images, clip_value_min=0, clip_value_max=1))) # keep values normalised by clipping
  #tensor = tensor.map(lambda images: (tf.cast(images, tf.uint8)))
  print(tensor)
  tensor = tensor.shuffle(1000)
  tensor = tensor.batch(32)
  tensor = tensor.prefetch(20)
  return tensor


train_dataset = x_train.apply(preprocessing)
test_dataset = x_test.apply(preprocessing)

print(train_dataset)


11501568/11490434 [==============================] - 0s 0us/step
<TensorSliceDataset shapes: (28, 28), types: tf.uint8>
<MapDataset shapes: (28, 28, 1), types: tf.float32>
<MapDataset shapes: (28, 28, 1), types: tf.float32>
<PrefetchDataset shapes: (None, 28, 28, 1), types: tf.float32>


#2.1 Convolutional Autoencoder

• The Autoencoder should consist of an encoder and a decoder, which can
be called independently. 6
• Encoder: The encoder should reduce the size of feature maps like a CNN.78
At the end of the encoder, flatten the feature maps and use a dense layer
to produce an embedding of a certain size. 9
• Decoder: The decoder takes the embedding from the encoder as input.
Use a dense layer to restore the dimensionality of the flattened feature
maps from the encoder and reshape the resulting vector into feature maps
again. Use upsampling or transposed convolutions to mirror your encoder.
As an output layer, use a convolutional layer with one filter and sigmoid
activation to produce an output image.


In [3]:
class Encoder(tf.keras.Model): 
  def __init__(self): 
    super(Encoder, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28, 28))
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(10, activation='softmax')

  @tf.function
  def call(self, input):
    x = self.conv1(input)
    x = self.flatten(x)
    x = self.dense(x)
    return x

In [4]:
model = Encoder()

out = model(train_dataset)

TypeError: ignored